In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Team Names: Ahmed Essam Ahmed Fawzy 
#            Amr Adel
#            Hussam Ashraf

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import ElasticNet
import matplotlib.pyplot as plt
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import RidgeCV
from sklearn.compose import ColumnTransformer
import xgboost
from sklearn.metrics import mean_squared_log_error

In [ ]:
def trans(x,l1=0.3,l2=0):
    if l1!=0:
        return ((x+l2)**l1-1)/l1
    else:
        return np.log(x+l2)
def rev_trans(x,l1=0.3,l2=0):
    return (x*l1+1)**(1/l1)-l2



In [ ]:
df = pd.read_csv("/kaggle/input/bike-sharing-demand/train.csv")

# Remove zero values of windspeed and humidity as they shouldn't be equal zero.

In [ ]:
df['windspeed'] = np.where(df['windspeed']==0.0,8.9981,df['windspeed'])

In [ ]:
df['humidity'] = np.where(df['humidity']==0.0,8,df['humidity'])

In [ ]:
df.describe().T

# EDA

In [ ]:
sns.pairplot(df)

# As Appeared the Temp and aTemp attributes are dependent so we cn drop any one of them.

In [ ]:
df.drop(columns='temp',inplace = True)

In [ ]:
df.hist(figsize = (20,20))

# The Hist of the attributes shows some skewness in some attributes which we can transform them using logging.

In [ ]:
df['windspeed'] = np.log(df['windspeed'])

In [ ]:
df.hist(figsize = (20,20))

In [ ]:
df['weather'].value_counts()

## The 4th category has only one value so we will add it to the third category.

In [ ]:
df['weather'] = np.where(df['weather']==4,3,df['weather'])

In [ ]:
df['weather'].value_counts()

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [ ]:
df['Hour'] = df['datetime'].dt.hour
df['DayOfWeek'] = df['datetime'].dt.dayofweek
df['Year'] = df['datetime'].dt.year
df['WeekOfYear'] = df['datetime'].dt.isocalendar().week
df['Month'] = df['datetime'].dt.month

In [ ]:
df = df.pivot_table(index = 'datetime')

In [ ]:
df['count'].resample('W').mean().plot() # asfreq method is used to convert a time series to a specified frequency. Here it is monthly frequency.
plt.title('Bike counts over time(Monthly frequency)')
plt.show()

In [ ]:
df['2011':'2011'].asfreq('D').plot(subplots=True, figsize=(20,24))
plt.title('All attributes in 2011')
plt.show()

# Adding new feature as the average count in season,month,day of weekand hour.

In [ ]:
hourMonthSeason = df.pivot_table(index=['season','Month','DayOfWeek','Hour'],values = 'count',aggfunc=np.mean).reset_index()
hourMonthSeason['y_SMH'] = hourMonthSeason['count'] 

In [ ]:
hourMonthSeason.info()

In [ ]:
df = pd.merge(df,hourMonthSeason[['season','Month','DayOfWeek','Hour','y_SMH']],on = ['season','Month','DayOfWeek','Hour'],how = 'left')

In [ ]:
df.info()

In [ ]:
X = df.copy()
X.drop(columns = ['WeekOfYear','casual', 'count', 'registered','holiday'],inplace = True)

In [ ]:
y_log=trans(df['count'])
X_train, X_eval, y_train, y_eval=train_test_split(X, y_log, test_size=0.3, random_state=42)

In [ ]:
xgb_best = xgboost.XGBRegressor(colsample_bytree= 0.7,
                                 learning_rate =0.03,
                                 max_depth= 5,
                                 min_child_weight= 0.4,
                                 n_estimators= 250,
                                 nthread= 4,
                                 objective= 'reg:linear',
                                 silent= 1,
                                 subsample= 0.7)


xgb_best.fit(X_train,y_train)
pred = xgb_best.predict(X_eval)
print(r2_score(y_eval,pred))
print(np.sqrt(mean_squared_error(y_eval,pred)))


# Handeling test data

In [ ]:
df_test = pd.read_csv("/kaggle/input/bike-sharing-demand/test.csv")

In [ ]:
df_test.info()

In [ ]:
df_test['windspeed'] = np.where(df_test['windspeed']==0.0,8.9981,df_test['windspeed'])
df_test['humidity'] = np.where(df_test['humidity']==0.0,8,df_test['humidity'])

In [ ]:
df_test.drop(columns='temp',inplace = True)
df_test['windspeed'] = np.log(df_test['windspeed'])
df_test['weather'] = np.where(df_test['weather']==4,3,df_test['weather'])

In [ ]:
df_test['datetime'] = pd.to_datetime(df_test['datetime'])
df_test['Hour'] = df_test['datetime'].dt.hour
df_test['DayOfWeek'] = df_test['datetime'].dt.dayofweek
df_test['Year'] = df_test['datetime'].dt.year
df_test['WeekOfYear'] = df_test['datetime'].dt.isocalendar().week
df_test['Month'] = df_test['datetime'].dt.month

In [ ]:
df_test = pd.merge(df_test,hourMonthSeason[['season','Month','DayOfWeek','Hour','y_SMH']],on = ['season','Month','DayOfWeek','Hour'],how = 'left')

In [ ]:
X_test = df_test.copy()
test_date = X_test['datetime'] 
X_test.drop(columns = ['WeekOfYear','datetime','holiday'],inplace = True)

In [ ]:
cols_list = X_train.columns.tolist()

In [ ]:
X_test = X_test[cols_list]

In [ ]:
y_test_predict=xgb_best.predict(X_test)
y_test_final = rev_trans(y_test_predict)
df_test['count']=y_test_final

In [ ]:
df_test[['datetime','count']]


In [ ]:
df_test[['datetime','count']].to_csv('/kaggle/working/submissionNoParams.csv', index=False)